In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import time

# Step 1: Load IBM Quantum Account using QiskitRuntimeService with your API token
service = QiskitRuntimeService(channel="ibm_quantum", token="api tokeb")

# Check if the service is loaded successfully
print("IBM Quantum account successfully loaded!")

# Step 2: Define the quantum circuit for generating random bits
def binary_to_password(binary_string, length):
    """
    Convert a binary string to a password.
    Each 8 bits represent one character.
    Replace non-printable characters with a random printable character.
    """
    printable_range = list(range(33, 127))  # ASCII values of printable characters ('!' to '~')
    password = ''
    
    for i in range(0, len(binary_string), 8):  # Split into 8-bit chunks
        byte = binary_string[i:i+8]
        char_code = int(byte, 2)
        
        # If the character is printable, add it; otherwise, map to a printable character
        if 33 <= char_code <= 126:
            password += chr(char_code)
        else:
            # Map non-printable characters to the printable range deterministically
            mapped_char = chr(printable_range[char_code % len(printable_range)])
            password += mapped_char
    
    return password


def generate_random_bits(num_bits):
    """
    Generate random bits using a quantum circuit with Qiskit's RNG.
    """
    num_qubits = num_bits  # One qubit per random bit
    qc = QuantumCircuit(num_qubits, num_qubits)  # Create the quantum circuit with qubits and classical register
    
    # Apply Hadamard gate to each qubit to create superposition
    for qubit in range(num_qubits):
        qc.h(qubit)
    
    # Measure all qubits into the classical register
    qc.measure(range(num_qubits), range(num_qubits))

    # Transpile the quantum circuit to match the backend's gate set
    backend = service.backends()[2]  # Use the second available backend (Kyiv)
    transpiled_qc = transpile(qc, backend)  # Transpile the circuit

    # Use the Sampler primitive to run the quantum circuit
    job = Sampler(backend).run([transpiled_qc])  # Wrap the circuit in a list and run
    
    # Wait for the job to finish (add timeout or retries to prevent infinite looping)
    max_retries = 5
    retries = 0
    result = None
    while retries < max_retries:
        try:
            result = job.result()
            print(f"Progress: Job completed successfully after {retries + 1} retries.")
            break
        except Exception as e:
            print(f"Error occurred: {e}. Retrying... (Attempt {retries + 1}/{max_retries})")
            retries += 1
            time.sleep(5)  # Delay before retrying
            if retries >= max_retries:
                raise Exception("Max retries reached, job failed to complete.")
    
    # Get the classical register name (this should match the register used in the circuit)
    classical_register = qc.cregs[0].name  # We only have one classical register
    
    # Access the counts for the classical register
    counts = result[0].data[classical_register].get_counts()
    #print(f"Counts: {counts}")  # Print the counts of the random bits generated

    return counts

# Step 3: Generate a quantum password
def generate_password(length=8):
    """
    Generate a random password using quantum randomness.
    """
    print(f"Generating quantum password of length {length}...")
    counts = generate_random_bits(length * 8)  # 8 bits per character
    
    # Extract the bitstring (you can choose the most frequent outcome or just take the first one)
    bit_string = max(counts, key=counts.get)  # Get the most frequent bit string
    print(f"Bit string selected: {bit_string}")  # Print the selected bit string
    password = binary_to_password(bit_string, length)
    return password

# Step 4: Generate and print the quantum password
password = generate_password(length=8)
print(f"Generated Quantum Password: {password}")


IBM Quantum account successfully loaded!
Generating quantum password of length 8...
Progress: Job completed successfully after 1 retries.
Bit string selected: 0110011111110100011110101001010110111000101010000100011110010111
Generated Quantum Password: gYzX{kGZ


In [4]:
import random
from scipy.stats import chisquare, entropy

def generate_prng_bits(num_bits):
    """
    Generate a random bit string using Python's pseudorandom number generator (PRNG).
    """
    return ''.join(str(random.randint(0, 1)) for _ in range(num_bits))

def test_randomness(bits, label):
    """
    Test the randomness of a bit string using statistical methods.
    """
    # Count occurrences of '0' and '1'
    counts = [bits.count('0'), bits.count('1')]
    chi = chisquare(counts)  # Perform Chi-Square Test
    entropy_value = entropy([counts[0], counts[1]], base=2)  # Calculate entropy

    print(f"{label} Randomness Test Results:")
    print(f" - Chi-Square p-value: {chi.pvalue}")
    print(f" - Entropy: {entropy_value}")
    return chi.pvalue, entropy_value

# Step 1: Generate Quantum Random Numbers
print("Generating Quantum Random Numbers...")
qrng_counts = generate_random_bits(100)  # Generate 100 quantum random bits
qrng_bit_string = ''.join(max(qrng_counts, key=qrng_counts.get) for _ in range(10))  # Create a bit string

# Step 2: Generate Pseudorandom Numbers
print("Generating Pseudorandom Numbers...")
prng_bit_string = generate_prng_bits(100)  # Generate 1000 pseudorandom bits

# Step 3: Perform Randomness Tests
qrng_results = test_randomness(qrng_bit_string, "Quantum RNG")
prng_results = test_randomness(prng_bit_string, "Pseudorandom RNG")

# Step 4: Display Results
print("\nRandomness Comparison:")
print(f"Quantum RNG - Chi-Square p-value: {qrng_results[0]}, Entropy: {qrng_results[1]}")
print(f"Pseudorandom RNG - Chi-Square p-value: {prng_results[0]}, Entropy: {prng_results[1]}")


Generating Quantum Random Numbers...
Progress: Job completed successfully after 1 retries.
Generating Pseudorandom Numbers...
Quantum RNG Randomness Test Results:
 - Chi-Square p-value: 0.00014780231033445398
 - Entropy: 0.9895875212220555
Pseudorandom RNG Randomness Test Results:
 - Chi-Square p-value: 0.10959858339911568
 - Entropy: 0.9814538950336537

Randomness Comparison:
Quantum RNG - Chi-Square p-value: 0.00014780231033445398, Entropy: 0.9895875212220555
Pseudorandom RNG - Chi-Square p-value: 0.10959858339911568, Entropy: 0.9814538950336537
